In [1]:

import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
import yfinance as yf
import time
from os import system, name
import matplotlib.pyplot as plt

s = yf.Ticker("MMYT")
opt = s.option_chain('2020-08-21')
calls, puts = opt.calls, opt.puts
print(int(calls['volume'].mean()))
calls['Fair Price'] = [i if i >= 0.01 else 0.01 for i in (calls['ask']+calls['bid'])/2]

def Call_Credit_Spread():
    
    c = calls[calls['volume']>=int(calls['volume'].mean())]
    l = c[['strike', 'Fair Price']]
    j = c[['strike', 'lastPrice']]
    if c['bid'].mean() != 0:
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
    else:
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

# best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}

    # First loop to check what to short (sell)   
    for short in n:
        
        # Second loop to check what we're buying (long)
        for long in n:
            
            # In a credit call spread the long Strike cannot be lower or equal to the short strike price
            if long<=short:
                continue
            else:
                
                # Declaring variables to work with
                strikeShort = short
                strikeLong = long
                
                premiumShort = n[short]
                premiumLong = n[long]
                
                # Figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = premiumShort - premiumLong
                
                #print("MaxReward = ", MaxReward)
                
                MaxRisk = (strikeLong - strikeShort - MaxReward)
                
                #print("MaxRisk = ", MaxRisk)
                
                # Cannot divide by 0.
                if MaxReward == 0:
                    continue
                
                # We don't want the reward to be in negative, which means that no matter what, we won't be profitting
                elif MaxReward>0:
                    
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
    # returning the final best trade available as a call credit spread              
    return print("Best Call Credit Spread: ", best_ratio)

Call_Credit_Spread()



13
Using Current Market Price
Best Call Credit Spread:  {'Short(Sell)': 15.0, 'Long(Buy)': 17.5, 'Risk/Reward Ratio': 1.5641025641025639, 'Short Premium': 1.8, 'Long Premium': 0.825, 'maxRisk': 1.525, 'maxReward': 0.9750000000000001}
